In [ ]:
# #Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install pandas

# NOTE: Before executing this ipynb, make sure you have done the following things:
### 1. Change the table name of green-db to green-db-backup (from your favorite sql client)

```sql
ALTER TABLE "green-db" RENAME TO "green-db-backup";
```

### 2. Redeploy workers

```bash
make workers-test-deploy
```

### 3. Insert old data into new Scraping Tables 

```sql
--DE
INSERT INTO "otto_DE"
	SELECT "id", "timestamp", "merchant" AS "source", "merchant", 'DE' AS country, "category", "url", "html", "page_type",
	  CASE 
	    WHEN meta_information->>'sex' IS NULL THEN 'UNCLASSIFIED'
		 ELSE upper(meta_information->>'sex')
		END
		AS gender,
	  CASE
	    WHEN meta_information"otto_DE"->>'sex' IS NULL THEN 'UNCLASSIFIED'
	    ELSE 'ADULT'
	  END
	  AS consumer_lifestage,
	  "meta_information"
	FROM otto;
	
INSERT INTO "amazon_DE"
	SELECT "id", "timestamp", "merchant" AS "source", "merchant", 'DE' AS country, "category", "url", "html", "page_type",
	  CASE 
	    WHEN meta_information->>'sex' IS NULL THEN 'UNCLASSIFIED'
		 ELSE upper(meta_information->>'sex')
		END
		AS gender,
	  CASE
	    WHEN meta_information->>'sex' IS NULL THEN 'UNCLASSIFIED'
	    ELSE 'ADULT'
	  END
	  AS consumer_lifestage,
	  "meta_information"
	FROM amazon;
	
INSERT INTO "zalando_DE"
	SELECT "id", "timestamp", "merchant" AS "source", "merchant", 'DE' AS country, "category", "url", "html", "page_type",
	  CASE 
	    WHEN meta_information->>'sex' IS NULL THEN 'UNCLASSIFIED'
		 ELSE upper(meta_information->>'sex')
		END
		AS gender,
	  CASE
	    WHEN meta_information->>'sex' IS NULL THEN 'UNCLASSIFIED'
	    ELSE 'ADULT'
	  END
	  AS consumer_lifestage,
	  "meta_information"
	FROM zalando;

--FR
INSERT INTO "asos_FR"
	SELECT "id", "timestamp", "merchant" AS "source", "merchant", 'FR' AS country, "category", "url", "html", "page_type",
	  CASE 
	    WHEN meta_information->>'sex' IS NULL THEN 'UNCLASSIFIED'
		 ELSE upper(meta_information->>'sex')
		END
		AS gender,
	  CASE
	    WHEN meta_information->>'sex' IS NULL THEN 'UNCLASSIFIED'
	    ELSE 'ADULT'
	  END
	  AS consumer_lifestage,
	  "meta_information"
	FROM asos;

INSERT INTO "hm_FR"
	SELECT "id", "timestamp", "merchant" AS "source", "merchant", 'FR' AS country, "category", "url", "html", "page_type",
	  CASE 
	    WHEN meta_information->>'sex' IS NULL THEN 'UNCLASSIFIED'
		 ELSE upper(meta_information->>'sex')
		END
		AS gender,
	  CASE
	    WHEN meta_information->>'sex' IS NULL THEN 'UNCLASSIFIED'
	    ELSE 'ADULT'
	  END
	  AS consumer_lifestage,
	  "meta_information"
	FROM hm;

INSERT INTO "zalando_FR"
	SELECT "id", "timestamp", 'zalando' AS "source", 'zalando' AS "merchant", 'FR' AS country, "category", "url", "html", "page_type",
	  CASE 
	    WHEN meta_information->>'sex' IS NULL THEN 'UNCLASSIFIED'
		 ELSE upper(meta_information->>'sex')
		END
		AS gender,
	  CASE
	    WHEN meta_information->>'sex' IS NULL THEN 'UNCLASSIFIED'
	    ELSE 'ADULT'
	  END
	  AS consumer_lifestage,
	  "meta_information"
	FROM zalando_fr;

INSERT INTO "amazon_FR"
	SELECT "id", "timestamp", 'amazon' AS "source", 'amazon' AS "merchant", 'FR' AS country, "category", "url", "html", "page_type",
	  CASE 
	    WHEN meta_information->>'sex' IS NULL THEN 'UNCLASSIFIED'
		 ELSE upper(meta_information->>'sex')
		END
		AS gender,
	  CASE
	    WHEN meta_information->>'sex' IS NULL THEN 'UNCLASSIFIED'
	    ELSE 'ADULT'
	  END
	  AS consumer_lifestage,
	  "meta_information"
	FROM amazon_fr;


--GB
INSERT INTO "zalando_GB"
	SELECT "id", "timestamp", 'zalando' AS "source", 'zalando' AS "merchant", 'GB' AS country, "category", "url", "html", "page_type",
	  CASE 
	    WHEN meta_information->>'sex' IS NULL THEN 'UNCLASSIFIED'
		 ELSE upper(meta_information->>'sex')
		END
		AS gender,
	  CASE
	    WHEN meta_information->>'sex' IS NULL THEN 'UNCLASSIFIED'
	    ELSE 'ADULT'
	  END
	  AS consumer_lifestage,
	  "meta_information"
	FROM zalando_uk;
```

### 4. Set autoincrements for all scraping tables

```sql
-- SET autoincrements
Select setval('"zalando_DE_id_seq"'::REGCLASS, (select max(id) FROM "zalando_DE"));
Select setval('"zalando_FR_id_seq"'::REGCLASS, (select max(id) FROM "zalando_FR"));
Select setval('"zalando_GB_id_seq"'::REGCLASS, (select max(id) FROM "zalando_GB"));

Select setval('"hm_FR_id_seq"'::REGCLASS, (select max(id) FROM "hm_FR"));
Select setval('"asos_FR_id_seq"'::REGCLASS, (select max(id) FROM "asos_FR"));
Select setval('"otto_DE_id_seq"'::REGCLASS, (select max(id) FROM "otto_DE"));

Select setval('"amazon_FR_id_seq"'::REGCLASS, (select max(id) FROM "amazon_FR"));
Select setval('"amazon_DE_id_seq"'::REGCLASS, (select max(id) FROM "amazon_DE"));
```

### 5. Add postgres passwords in the following cell

### 6. Run this ipynb

**6.1 load scraping tables**

**6.2 load old greendb table** 

**6.3 merge scraping and greendb (except asos)**

**6.4 join asos exclusively**

**6.5 combine joined asos and all others**

**6.6 insert merged data into new greendb**

### 7. Set autoincrements for green-db table

```sql
Select setval('"green-db_id_seq"'::REGCLASS, (select max(id) FROM "green-db"));
```

# 6. RUN this ipynb
## 6.1 load scraping tables

In [ ]:
import os
os.environ["POSTGRES_SCRAPING_USER"] = 'scraping'
os.environ["POSTGRES_SCRAPING_PASSWORD"] = '' # TODO
os.environ["POSTGRES_SCRAPING_HOST"] = '127.0.0.1'
os.environ["POSTGRES_SCRAPING_PORT"] = '5432'

os.environ["POSTGRES_GREEN_DB_USER"] = "green-db"
os.environ["POSTGRES_GREEN_DB_PASSWORD"] = "" # TODO
os.environ["POSTGRES_GREEN_DB_HOST"] = "localhost"
os.environ["POSTGRES_GREEN_DB_PORT"] = "5432"

from database.connection import Scraping
import pandas as pd
import numpy as np

In [ ]:
from sqlalchemy import ARRAY, BIGINT, INTEGER, JSON, NUMERIC, TEXT, TIMESTAMP, VARCHAR, Column

from datetime import datetime
from enum import Enum
from typing import List, Optional

from pydantic import BaseModel, conint, conlist


class PageType(str, Enum):
    SERP = "SERP"
    PRODUCT = "PRODUCT"


class ScrapedPageNoHTML(BaseModel):
    timestamp: datetime
    source: str
    merchant: str
    country: str
    url: str
    #page_type: PageType
    category: str
    gender: str
    consumer_lifestage: str
    #meta_information: dict

    class Config:
        orm_mode = True
        use_enum_values = True

In [ ]:
from database.connection import Scraping, Connection
from core.domain import ScrapedPage
from message_queue import MessageQueue
from typing import Iterator


class Scraping2(Scraping):      
    def get_scraped_products_no_html(self, batch_size: int = 1000) -> Iterator[ScrapedPage]:
        """
        Fetch all `ScrapedPage`s.

        Args:        
            batch_size (int, optional): How many rows to fetch simultaneously. Defaults to 1000.

        Yields:
            Iterator[ScrapedPage]: Iterator over the domain object representations
        """
        with self._session_factory() as db_session:
            query = db_session.query(self._database_class.id, 
                                     self._database_class.timestamp, 
                                     self._database_class.source,
                                     self._database_class.merchant, 
                                     self._database_class.country, 
                                     self._database_class.url, 
                                     #self._database_class.page_type, 
                                     self._database_class.category, 
                                     self._database_class.gender, 
                                     self._database_class.consumer_lifestage,
                                     #self._database_class.meta_information,
                                    ).filter(
                self._database_class.page_type == 'PRODUCT'
            )
            return (ScrapedPageNoHTML.from_orm(row) for row in query.all())

In [ ]:
name_2_db = {
    "zalando_DE": Scraping2("zalando_DE"),
    "zalando_GB": Scraping2("zalando_GB"),
    "zalando_FR": Scraping2("zalando_FR"),
    "otto_DE": Scraping2("otto_DE"),      
    "hm_FR": Scraping2("hm_FR"),
    "amazon_DE": Scraping2("amazon_DE"),
    "amazon_FR": Scraping2("amazon_FR"),
    "asos_FR": Scraping2("asos_FR"),
}

# name_2_db = {
#     "zalando": Scraping2("zalando"),
#     "zalando_uk": Scraping2("zalando_uk"),
#     "zalando_fr": Scraping2("zalando_fr"),
#     "otto": Scraping2("otto"),      
#     "hm": Scraping2("hm"),
#     "amazon": Scraping2("amazon"),
#     "amazon_fr": Scraping2("amazon_fr"),
#     "asos": Scraping2("asos"),
# }

In [ ]:
all_scraping_products = pd.DataFrame()

for name, db in name_2_db.items():
    products_iterator = db.get_scraped_products_no_html()
    products = pd.DataFrame([product.__dict__ for product in products_iterator])
    
    all_scraping_products = pd.concat([all_scraping_products, products])
    
all_scraping_products.shape

In [ ]:
# if kernel dies it might be good to have stored the df locally
#all_products.to_parquet("../data/products-meta-2022-06-13.parquet", index=False) #TODO

In [ ]:
all_scraping_products.isna().sum()

In [ ]:
all_scraping_products["merchant"].value_counts()

In [ ]:
all_scraping_products.head(n=2)

## 6.2 load old greendb table

In [ ]:
from database.tables import GreenDBBaseTable, __TableMixin
from sqlalchemy import ARRAY, BIGINT, INTEGER, JSON, NUMERIC, TEXT, TIMESTAMP, VARCHAR, Column

from datetime import datetime
from enum import Enum
from typing import List, Optional

from pydantic import BaseModel, conint, conlist

from core.sustainability_labels import create_CertificateType

CertificateType = create_CertificateType()


class GreenDBTableOld(GreenDBBaseTable, __TableMixin):
    """
    Defines the GreenDB columns.

    Args:
        GreenDBBaseTable ([type]): `sqlalchemy` base class for the GreenDB database
        __TableMixin ([type]): Mixin that implements some convenience methods
    """

    __tablename__ = "green-db-backup" #TODO

    id = Column(INTEGER, nullable=False, autoincrement=True, primary_key=True)
    timestamp = Column(TIMESTAMP, nullable=False)
    merchant = Column(TEXT, nullable=False)
    category = Column(TEXT, nullable=False)
    url = Column(TEXT, nullable=False)
    name = Column(TEXT, nullable=False)
    description = Column(TEXT, nullable=False)
    brand = Column(TEXT, nullable=False)
    sustainability_labels = Column(ARRAY(TEXT), nullable=False)  # TODO foreign keys to labels
    price = Column(NUMERIC, nullable=False)
    currency = Column(TEXT, nullable=False)
    image_urls = Column(ARRAY(TEXT), nullable=False)

    color = Column(TEXT, nullable=True)
    size = Column(TEXT, nullable=True)

    gtin = Column(BIGINT, nullable=True)
    asin = Column(TEXT, nullable=True)
    
class CurrencyType(str, Enum):
    EUR = "EUR"
    GBP = "GBP"
    
class Productold(BaseModel):
    timestamp: datetime
    url: str
    merchant: str
    category: str
    name: str
    description: str
    brand: str
    sustainability_labels: conlist(CertificateType, min_items=1)  # type: ignore
    price: float
    currency: CurrencyType
    image_urls: List[str]

    color: Optional[str]
    size: Optional[str]

    # int, source: https://support.google.com/merchants/answer/6219078?hl=en
    gtin: Optional[int]

    # str because alpha numeric
    # source: https://en.wikipedia.org/wiki/Amazon_Standard_Identification_Number
    asin: Optional[str]

    class Config:
        orm_mode = True
        use_enum_values = True

In [ ]:
from database.connection import Connection
from core.constants import DATABASE_NAME_GREEN_DB
from database.tables import SustainabilityLabelsTable
from typing import Iterator
from core.domain import Product

class GreenDBold(Connection):
    def __init__(self) -> None:
        """
        `Connection` for the GreenDB.
        Automatically pre-populates the sustainability labels table.
        """
        super().__init__(GreenDBTableOld, DATABASE_NAME_GREEN_DB)
    
    def get_all_products(self, batch_size: int = 10000) -> Iterator[Product]:
        """
        Fetch all `Products`.

        Args:            
            batch_size (int, optional): How many rows to fetch simultaneously. Defaults to 1000.

        Yields:
            Iterator[ScrapedPage]: Iterator over the domain object representations
        """
        with self._session_factory() as db_session:
            query = db_session.query(self._database_class)
            return (Productold.from_orm(row) for row in query.all())

In [ ]:
greenDBold = GreenDBold()

products_iterator = greenDBold.get_all_products()
greenDBproducts = pd.DataFrame([product.__dict__ for product in products_iterator])
greenDBproducts.shape

In [ ]:
greenDBproducts.head(n=2)


## 6.3 merge scraping and greendb (except asos)

In [ ]:
#store asos seperately
scraping_asos = all_scraping_products[all_scraping_products["merchant"]=="asos"]

In [ ]:
#prep index for joining

tuples = list(zip(all_scraping_products['timestamp'], all_scraping_products['url'], all_scraping_products['category']))
index = pd.MultiIndex.from_tuples(tuples, names=["timestamp", "url", "category"])

all_scraping_products.index = index

In [ ]:
# remove unwanted columns
all_scraping_products = all_scraping_products[["source", "country", "gender", "consumer_lifestage"]]

In [ ]:
all_scraping_products.head(n=2)

In [ ]:
joined = greenDBproducts.join(all_scraping_products, on=['timestamp', 'url', 'category'])
joined.head(n=2)

In [ ]:
# remove duplicates due to joining

# the shops might list products in multiple categories which we have just assigned to one category
# this creates duplicate extracted products
# so joining the df, using just timestamp, url and catgeory (green-db specific category) 
# we can not fully map instancs from scraping table to green-db table
# but for including gender information this is not necessary.
# If we have multiple products with same url and different categories assigned to different genders
# the extracted products are just duplicates of each other, 
# because the product is basically a UNISEX product assigned to multiple categories

joined_deduplicated = joined.drop_duplicates(subset=['timestamp', 'url', 'source', 'merchant', 'category', 'gender', 'consumer_lifestage'])
joined_deduplicated.shape

In [ ]:
joined_deduplicated.merchant.value_counts()

In [ ]:
#check null values. Only color, size, gtin and asin should have null values!
# null values in source, country gender and consumer_lifestage are due to asos
joined_deduplicated.isna().sum()

In [ ]:
# amount of None encapsulated as string (will be transformed later)
joined_deduplicated[joined_deduplicated["size"]=="None"].shape

In [ ]:
#remove asos
joined_deduplicated = joined_deduplicated[joined_deduplicated["merchant"]!="asos"]
joined_deduplicated.shape

In [ ]:
# only color, size, gtin, asin should be null
joined_deduplicated.isna().sum()

## 6.4 join asos exclusively

In [ ]:
greendb_asos = greenDBproducts[greenDBproducts["merchant"]=="asos"]

In [ ]:
scraping_asos["product_id"] = scraping_asos["url"].apply(lambda x: x.split("/")[-1].split("?")[0])
scraping_asos["product_id"] = scraping_asos["product_id"].astype('int64')

greendb_asos["product_id"] = greendb_asos["url"].apply(lambda x: x.split("/")[-1])
greendb_asos["product_id"] = greendb_asos["product_id"].astype('int64')

In [ ]:
#prep product index
tuples = list(zip( greendb_asos['product_id'], greendb_asos['timestamp'], greendb_asos['category']))
index = pd.MultiIndex.from_tuples(tuples, names=["timestamp", "url", "category"])
#greendb_asos["id"] = greendb.index
greendb_asos.index = index

#prep index for joining
tuples = list(zip(scraping_asos['product_id'],scraping_asos['timestamp'], scraping_asos['category']))
index = pd.MultiIndex.from_tuples(tuples, names=["timestamp", "url", "category"])
#scraping["id"] = scraping.index
scraping_asos.index = index

In [ ]:
#remove unwanted columns
scraping_asos = scraping_asos[["source", "country", "gender", "consumer_lifestage"]]

In [ ]:
asos_joined = greendb_asos.join(scraping_asos, how="left")

In [ ]:
asos_joined.isna().sum()

In [ ]:
#remove duplicates
asos_joined_deduplicated = asos_joined.drop_duplicates(subset=['timestamp', 'url', 'merchant', 'category', 'gender'])
asos_joined_deduplicated.shape

## 6.5 combine joined asos and all others

In [ ]:
joined_deduplicated = pd.concat([asos_joined_deduplicated, joined_deduplicated])
joined_deduplicated.shape

## 6.6 insert merged data into new greendb

In [ ]:
from database.connection import GreenDB
from core.domain import Product

class GreenDBnew(GreenDB):
    def write_df(self, df):
        """
        Writes a `dataframe` into the database.        
        """
        with self._session_factory() as db_session:
            i = 0
            df_len = len(df)
            for index, product in df.iterrows():
                i += 1
                try:                    
                    db_object = self._database_class(**Product.parse_obj(product).dict())
                    db_session.add(db_object)
                except Exception as e:
                    print(f"error for product with index: {index}")
                    print(e)
                #commit every 1000 products and at the end
                if (i % 1000 == 0) or (i == df_len):
                    db_session.commit()
                    print(f"Commited {i} products")


greenDBnew = GreenDBnew()

In [ ]:
# transform merchant
merchant_to_new_merchant = {
    "asos": "asos",
    "amazon": "amazon",
    "amazon_fr": "amazon",
    "zalando": "zalando",
    "zalando_fr": "zalando",
    "zalando_uk": "zalando",
    "otto": "otto",
    "hm": "hm"
}

joined_deduplicated["merchant"] = joined_deduplicated["merchant"].apply(lambda x: merchant_to_new_merchant.get(x))

# transform color
joined_deduplicated["colors"] = joined_deduplicated["color"].apply(lambda x: [x] if x else None)

# transform size
joined_deduplicated["sizes"] = joined_deduplicated["size"].apply(lambda sizes: None if sizes == 'None' or sizes is None else sizes.split(", "))

# need to convert NA to None, otherwise pydantic throws errors
joined_deduplicated = joined_deduplicated.replace({np.nan: None})

In [ ]:
# test pydantic product parsing
Product.parse_obj(joined_deduplicated.iloc[0])

In [ ]:
greenDBnew.write_df(joined_deduplicated)

In [ ]:
print(f"Dataframe size is: {len(joined_deduplicated)}")

Check product count in database and compare with dataframe size
```sql
Select Count(id) from "green-db";
```

If everything went well and the tables look good, you can delete the old tables e.g. via:

```sql
DROP TABLE "amazon", "asos", "otto", "zalando", "zalando_fr", "zalando_uk";
DROP TABLE "green-db-backup";
```

Do not forget to set the autoincrement for green-db (see step 7)